# Imports

In [1]:
import os
import glob
import pandas as pd
import shutil
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import torch
#from utils.utils import plot_results

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
images_dir = './raw/images'
train_dir = './dataset/train/images'
val_dir = './dataset/val/images'
test_dir = './dataset/test/images'
test_df = pd.read_csv('./raw/test.csv')

# Detection

In [11]:
#!python yolov5/detect.py --weights yolov5/runs/train/exp2/weights/best.pt --img 640 --conf 0.4 --source dataset/test/images

In [75]:
if not os.path.exists('./outputs'):
    os.mkdir('outputs')

weight_path = 'yolov5/runs/train/exp/weights/best.pt'
model = torch.hub.load('yolov5', 'custom', path=weight_path, source='local')
final_output = pd.DataFrame()


for image in test_df.image_path.unique():
    image_path = os.path.join(test_dir,image)
    results = model(image_path)
    output = results.pandas().xyxy[0]
    output['image_path'] = image
    final_output = pd.concat([final_output, output])

final_output.to_csv('./outputs/results_conf_shuffil.csv', index=False, encoding='utf-8-sig')

YOLOv5 🚀 v7.0-72-g064365d Python-3.8.10 torch-1.13.1+cu117 CUDA:0 (NVIDIA A100-SXM4-40GB, 40396MiB)

Fusing layers... 
Model summary: 267 layers, 46162128 parameters, 0 gradients, 107.8 GFLOPs
Adding AutoShape... 


# Detect with low conf

In [76]:
print('Results pics:', str(final_output.image_path.nunique()), final_output.shape )
print('Testset pics:', str(test_df.image_path.nunique()) )

Results pics: 1857 (5021, 8)
Testset pics: 2092


In [77]:
def tesl_low_conf(test_df, last_df, model, conf, keep ):
    not_predict_pics = test_df[~test_df.image_path.isin(last_df.image_path.unique())]
    val_output = pd.DataFrame()
    
    model.conf = conf
    
    for image in not_predict_pics.image_path.unique():
        results = model(os.path.join(test_dir,image))
        output = results.pandas().xyxy[0]
        output['image_path'] = image
        val_output = pd.concat([val_output, output])
    
    output = pd.DataFrame()
    for image in val_output.image_path.unique():
        selected = val_output[val_output.image_path == image].sort_values(by='confidence', ascending=False).head(keep)
        output = pd.concat([output, selected])
    
    output = pd.concat([output, last_df])
    print('output shape:' , output.shape)
    print('output images:' , output.image_path.nunique())
    
    return output
    

In [78]:
final_output.describe()

,xmin,ymin,xmax,ymax,confidence
count,5021.000000,5021.000000,5021.000000,5021.000000,5021.000000
mean,374.455001,197.766799,580.789183,317.117364,0.585879
std,274.884546,81.227119,271.788520,98.401234,0.168587
min,0.000000,0.000000,29.221693,27.773411,0.250019
25%,119.565788,146.249435,352.399963,246.475525,0.449159
50%,361.889587,189.865311,596.428833,299.797852,0.610979
75%,610.058838,242.575684,826.921265,376.172974,0.728731
max,926.057129,493.751282,960.000000,540.000000,0.901912


In [91]:
val_output = tesl_low_conf(test_df, final_output, model, 0.001,1)

output shape: (5256, 8)
output images: 2092


In [92]:
val_output.xmax = val_output.xmax.astype(int) 
val_output.xmin = val_output.xmin.astype(int)  
val_output.ymax = val_output.ymax.astype(int) 
val_output.ymin = val_output.ymin.astype(int) 


val_output.to_csv('./outputs/val_final_conf_shuffle.csv', index=False, encoding='utf-8-sig')
val_output = val_output[['class','image_path','name','xmax','xmin','ymax','ymin']]
val_output.to_csv('./outputs/final_results_shuffle.csv', index=False, encoding='utf-8-sig')

In [93]:
def object_detection(df, path=test_dir, change_width=False, image_path=False):
    
    if image_path != False:
        select_df = df[df.image_path == image_path].reset_index()
    else:
        df = df.sample(frac=1)
        image_path = df.iloc[0].image_path
        select_df = df[df.image_path == image_path].reset_index()
        
    print(image_path)
    image = cv2.cvtColor(cv2.imread(os.path.join(path, image_path)), cv2.COLOR_BGR2RGB)
    for i in select_df.index:
        xmin = select_df.iloc[i].xmin #* 2
        ymin = select_df.iloc[i].ymin #* 2
        xmax = select_df.iloc[i].xmax #* 2
        ymax = select_df.iloc[i].ymax #* 2
        label = select_df.iloc[i]['name']
        
        height = image.shape[0]
        width = image.shape[1]
        if change_width == True:
            width = 960
            image = cv2.resize(image, (width, height), interpolation = cv2.INTER_AREA)
            xmax = xmax / 2
            xmin = xmin / 2

        print('class:' , select_df.iloc[i]['name'])
        print('xmin:' , xmin)
        print('ymin:' , ymin)
        print('xmax:' , xmax)
        print('ymax:' , ymax)
        print('----------------------------')
            
        cv2.rectangle(image,
                      (int(xmin) , int(ymin) ),
                      (int(xmax) , int(ymax) ),
                      (0,255,0), thickness=2)

        ((label_width, label_height), _) = cv2.getTextSize(label, fontFace=cv2.FONT_HERSHEY_PLAIN, 
        fontScale=1.75, thickness=2)

        cv2.rectangle(
      image,
      (int(xmin) , int(ymin)),
      (int(xmin + label_width + label_width * 0.05), int(ymin + label_height + label_height * 0.25)),
      color=(255, 0, 0),
      thickness=cv2.FILLED
    )

        cv2.putText(
      image,
      label,
      org=(int(xmin), int(ymin + label_height + label_height * 0.25)), 
      fontFace=cv2.FONT_HERSHEY_PLAIN,
      fontScale=1.75,
      color=(255, 255, 255),
      thickness=2 )

    plt.imshow(image)
    plt.show()

            
